### Imports

In [1]:
import mlflow
from mlflow import MlflowClient
from mlflow.types.schema import Schema, ColSpec
from mlflow.types import ParamSchema, ParamSpec
from mlflow.models import ModelSignature
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import json
import os

#### Data

In [2]:
with open('shakespeare.txt','r',encoding='utf8') as f:
    text = f.read()

all_characters = set(text)

### Loading Model

In [3]:
class CharModel(nn.Module):
    
    def __init__(self, decoder, encoder, all_chars, num_hidden=256, num_layers=4,drop_prob=0.5, use_gpu=False):
        super().__init__()
        self.drop_prob = drop_prob
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.use_gpu = use_gpu
        
        self.all_chars = all_chars
        self.decoder = torch.load(decoder)
        self.encoder = torch.load(encoder)
        
        self.lstm = nn.LSTM(len(self.all_chars), num_hidden, num_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))
      
    
    def forward(self, x, hidden):
        lstm_output, hidden = self.lstm(x, hidden)       
        drop_output = self.dropout(lstm_output)
        drop_output = drop_output.contiguous().view(-1, self.num_hidden)
        final_out = self.fc_linear(drop_output)
        
        return final_out, hidden
    
    
    def hidden_state(self, batch_size):
        if self.use_gpu:
            hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda(),
                     torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda())
        else:
            hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden),
                     torch.zeros(self.num_layers,batch_size,self.num_hidden))
        
        return hidden


# MLFlow - Register Model

In [10]:
class RNNModel(mlflow.pyfunc.PythonModel):
    
    def load_context(self, context):
        self.model = CharModel(
                        all_chars=all_characters,
                        num_hidden=512,
                        num_layers=3,
                        drop_prob=0.5,
                        use_gpu=False,
                        decoder=context.artifacts['decoder'],
                        encoder=context.artifacts['encoder']
                                           
                    )


        self.model.load_state_dict(torch.load(context.artifacts['model_state_dict']))
        self.model.eval()

    def one_hot_encoder(self, encoded_text, num_uni_chars):
        one_hot = np.zeros((encoded_text.size, num_uni_chars))
        one_hot = one_hot.astype(np.float32)
        one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()] = 1.0
        one_hot = one_hot.reshape((*encoded_text.shape, num_uni_chars))
        
        return one_hot

    def predict_next_char(self, char, hidden=None, k=3):
        encoded_text = self.model.encoder[char]
        encoded_text = np.array([[encoded_text]])
        encoded_text = self.one_hot_encoder(encoded_text, len(self.model.all_chars))
        inputs = torch.from_numpy(encoded_text)
        inputs = inputs.cpu()
            
        hidden = tuple([state.data for state in hidden])
        lstm_out, hidden = self.model(inputs, hidden)    
        probs = F.softmax(lstm_out, dim=1).data
        probs = probs.cpu()

        
        probs, index_positions = probs.topk(k)        
        index_positions = index_positions.numpy().squeeze()
        probs = probs.numpy().flatten()
        probs = probs/probs.sum()
        char = np.random.choice(index_positions, p=probs)
    
        return self.model.decoder[char], hidden

    def generate_text(self, seed, size, k=3):

        self.model.cpu()
            
        self.model.eval()
        output_chars = [c for c in seed]
        hidden = self.model.hidden_state(1)
        
        for char in seed:
            char, hidden = self.predict_next_char(char, hidden, k=k)
    
        output_chars.append(char)
        for i in range(size):
            char, hidden = self.predict_next_char(output_chars[-1], hidden, k=k)
            output_chars.append(char)
            
        return ''.join(output_chars)
            
        
    def predict(self, context, model_input):
        initial_word = model_input['initial_word'][0]
        size = model_input['size'][0]
        output = self.generate_text(seed=initial_word, size=size)
        
        return output

    @classmethod
    def log_model(cls, model_state_dict, decoder, encoder, demo_folder="demo"): 
        input_schema = Schema(
            [
                ColSpec("string", "initial_word"),
                ColSpec("long", "size")
            ]
        )

        output_schema = Schema(
            [
                ColSpec("string", "generated_text")
            ]
        )
      
        signature = ModelSignature(inputs=input_schema, outputs=output_schema)
             
        requirements = [
            "torch",
            "numpy"
        ]
        mlflow.pyfunc.log_model(
            model_state_dict,
            python_model=cls(),
            artifacts={"model_state_dict": model_state_dict, 'decoder': decoder, 'encoder': encoder, "demo": demo_folder},
            signature=signature,
            pip_requirements=requirements
        )

In [11]:
mlflow.set_experiment(experiment_name='Shakespeare Text Generation')

<Experiment: artifact_location='/phoenix/mlflow/161459198344494027', creation_time=1733920341445, experiment_id='161459198344494027', last_update_time=1733920341445, lifecycle_stage='active', name='Shakespeare Text Generation', tags={}>

In [12]:
model_state_dict = 'models/dict_torch_rnn_model.pt'

In [13]:
register_name = 'Shakespeare_Model'

In [14]:
with mlflow.start_run(run_name='Shakespeare_main') as run:
    print(f"Run's Artifact URI: {run.info.artifact_uri}")
    RNNModel.log_model(model_state_dict, 'models/decoder.pt', 'models/encoder.pt')
    mlflow.register_model(model_uri = f"runs:/{run.info.run_id}/{model_state_dict}", name=register_name)

Run's Artifact URI: /phoenix/mlflow/161459198344494027/58a4811f783146c5a62e9e8d96bb78d0/artifacts


Registered model 'Shakespeare_Model' already exists. Creating a new version of this model...
Created version '3' of model 'Shakespeare_Model'.


In [9]:
client = mlflow.MlflowClient()
model_metadata = client.get_latest_versions(register_name, stages=["None"])
latest_model_version = model_metadata[0].version
latest_model_version

/tmp/ipykernel_235/863823821.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  model_metadata = client.get_latest_versions(register_name, stages=["None"])


2

#### Testing registered model

In [41]:
model = mlflow.pyfunc.load_model(model_uri=f"models:/{register_name}/{latest_model_version}")
print(model.predict({"initial_word": 'Love ', "size": 100}))

<bound method Module.state_dict of CharModel(
  (lstm): LSTM(84, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_linear): Linear(in_features=512, out_features=84, bias=True)
)>
Love to him; and
    I have seen him this, though I have sent my heart
    And we that bring the willing f
